In [ ]:
"""
Algorithm 1 LEO-Based EC Algorithm
1: Begin
2: // Initialization
3: g←0 ; // the generation index
4: Initialize population Xg and evaluate the fitness;
5: Initialize the weights of ANN randomly;
6: Initialize arch as an empty set; // to store solution pairs
7: While stop criteria not satisfied Do
8:		g←g+ 1;
9:		// Individual Evolution
10:		Sample r uniformly on [0,1];
11:		If g> 1 and r< lp Then
12:			newX ← Evolve Xg by learning-aided evolutionary operator;
13:		Else
14:			// use operators in traditional EC, e.g., PSO or DE
15:			newX ← Evolve Xg by traditional evolutionary operator;
16:		End If
17:		Evaluate the fitness of individuals in newX;
18:		// Selection
19:		Xg+1← selection among Xg and newX;
20:		// SEP Collection
21:		For each individual i in Xg+1 Do
22:			If Xg+1,i is better than Xg,i Then
23:				Add (Xg,i , Xg+1,i ) in arch;
24:			End If
25:		End For
26:		If number of SEPs in arch > arch_size Then
27:			arch ← the newest arch_size solution pairs;
28:		End If
29:		// Learning System Update
30:		Train the ANN with all data in arch for one epoch;
31:	End While
32:End
"""

In [47]:
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.problems import get_problem
from pymoo.core.population import Population
from pymoo.optimize import minimize
from keras.layers import Dense
from keras.models import Sequential
from pymoo.core.evaluator import Evaluator
from pymoo.indicators.igd import IGD
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.operators.mutation.pm import PM
from pymoo.operators.crossover.sbx import SBX
from pymoo.visualization.scatter import Scatter
import random

In [48]:
def normalize(data):
    min_val = -100
    max_val = 100
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data


def denormalize(data):
    min_val = -100
    max_val = 100
    denormalized_data = (data * (max_val - min_val)) + min_val
    return denormalized_data

In [49]:
def generate_model(n_var):
    model = Sequential()
    model.add(Dense((3*n_var), input_dim=n_var, activation='sigmoid'))
    model.add(Dense(n_var, activation='sigmoid'))

    model.compile(loss='mse',
                  optimizer='adam', metrics=['accuracy'])
    return model

In [50]:
def dominates(p, q):
    return np.all(p <= q) and np.any(p < q)

In [51]:

def traditional_nsga(problem, pop_size=1000, generation=1000, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    X = np.random.uniform(l_bound, h_bound, (pop_size, n_var))
    pop = Population.new("X", X)
    Evaluator().eval(problem, pop)

    algorithm = NSGA2(pop_size=pop_size, crossover=SBX(eta=15, prob=0.9),
                      mutation=PM(eta=20, prob=0.5), sampling=pop)
    res = minimize(problem, algorithm, ("n_gen", generation),
                   verbose=verbose)

    return res.X, res.F, res

In [52]:
n_obj = 10
n_var = 100
l_bound = -100
h_bound = 100
problem = get_problem('dtlz2', n_var=n_var, n_obj=n_obj)

problem.xl = np.array([-100 for _ in range(100)])
problem.xu = np.array([100 for _ in range(100)])

pop_traditional, pareto_traditional, res = traditional_nsga(
    problem, pop_size=100, generation=1000,
    n_var=100, l_bound=-100, h_bound=100, verbose=True)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        0 |     76 |             - |             -
     2 |      100 |    100 |  0.2904058168 |         ideal
     3 |      200 |    100 |  0.3090875111 |         ideal
     4 |      300 |    100 |  0.1256284374 |         ideal
     5 |      400 |    100 |  0.3014485593 |         ideal
     6 |      500 |    100 |  0.1146048223 |         nadir
     7 |      600 |    100 |  0.2045618835 |         ideal
     8 |      700 |    100 |  0.0554762609 |         ideal
     9 |      800 |    100 |  0.0261503966 |         ideal
    10 |      900 |    100 |  0.0640799060 |         ideal
    11 |     1000 |    100 |  0.0489384240 |         ideal
    12 |     1100 |    100 |  0.0044014140 |         ideal
    13 |     1200 |    100 |  0.0240510075 |         ideal
    14 |     1300 |    100 |  0.1198055604 |         nadir
    15 |     1400 |    100 |  0.0886041863 |         ideal
    16 |     1500 |    100 |  0.0467299773 |         ide

In [53]:
ref_dirs = get_reference_directions("das-dennis", n_obj, n_partitions=12)
pf_true = problem.pareto_front(ref_dirs)

igd = IGD(pf_true)

In [54]:
# Scatter().add(pareto_traditional).show()

In [55]:

def leo_based_nsta(problem, pop_size=100, generation=100, lp=0.1, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    pop = np.random.uniform(l_bound, h_bound, (pop_size, problem.n_var))

    fit = problem.evaluate(pop)
    model = generate_model(n_var)

    archive_x = []
    archive_y = []

    for g in range(generation):
        if g > 0 and random.random() < lp:
            pred = model.predict(normalize(pop), verbose=verbose)
            pop_new = denormalize(pred)
            fit_new = problem.evaluate(pop_new)
        else:
            algorithm = NSGA2(pop_size=pop_size, crossover=SBX(eta=15, prob=0.9),
                              mutation=PM(eta=20, prob=0.5), sampling=pop)
            res = minimize(problem, algorithm, ("n_gen", 1), verbose=False)
            pop_new = res.pop.get("X")
            fit_new = res.pop.get("F")
        for i in range(pop_size):
            if dominates(fit_new[i], fit[i]):
                archive_x.append(pop[i])
                archive_y.append(pop_new[i])
        pop = pop_new
        fit = fit_new
        if len(archive_x) > 0:
            archive_x = archive_x[-100:]
            archive_y = archive_y[-100:]
            model.fit(normalize(np.array(archive_x)), normalize(np.array(
                archive_y)), epochs=1, verbose=verbose)
        print(f"gen:  {g}, archive size: {len(archive_x)}", end='       \r')
    return pop, fit

In [56]:
n_obj = 10
n_var = 100
l_bound = -100
h_bound = 100

problem = get_problem('dtlz2', n_var=n_var, n_obj=n_obj)
problem.xl = np.array([-100 for _ in range(100)])
problem.xu = np.array([100 for _ in range(100)])

pop_leo, pareto_leo = leo_based_nsta(
    problem, pop_size=100, generation=1000,
    lp=0.1, n_var=n_var, l_bound=l_bound, h_bound=h_bound, verbose=False)

In [57]:
print("traditional: ", igd(pareto_traditional))
print("leo-based: ", igd(pareto_leo))

traditional:  85291098.27078137
leo-based:  184426762.79998508


In [58]:
# Scatter().add(pareto_leo).show()